# In Play Data And Match With Betfair Live Data

In [1]:
import requests
import urllib
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm_notebook
import datetime
import importlib
import config
importlib.reload(config)
from config import username, password, application, dbpw
import logging

import matplotlib.pyplot as plt
import seaborn as sns

import pymysql
import sqlalchemy

import xgboost as xgb
import statsmodels.api as sm

from Levenshtein import distance as levenshtein_distance

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Sporting Life Live Data

In [3]:
url = 'https://www.sportinglife.com/football/fixtures-results/live'
livejson = urllib.request.urlopen(url).read()
soup = BeautifulSoup(livejson)
soup_find = soup.body.find(attrs={"type": "application/json"})
soup_json = json.loads(soup_find.text)

In [4]:
matches_data = []
goals_data = []

matches = soup_json.get('props', {}).get('pageProps', {}).get('matches', [])
for m in matches:
    match_ref = m.get('match_reference', {}).get('id')
    match_date = m.get('match_date')
    match_time = m.get('match_time')

    matches_data.append(
        [
            match_ref, match_date, match_time,
            m.get('state'),
            m.get('match_type'), 
            m.get('competition', {}).get('competition_reference', {}).get('id'),
            m.get('competition', {}).get('name'),
            str(m.get('round')),
            m.get('legs'),
            m.get('leg'),
            m.get('team_score_a', {}).get('team', {}).get('team_reference', {}).get('id'),
            m.get('team_score_a', {}).get('team', {}).get('name'),
            m.get('team_score_a', {}).get('team', {}).get('short_name'),
            m.get('team_score_a', {}).get('score', [])[0].get('score'),
            m.get('team_score_b', {}).get('team', {}).get('team_reference', {}).get('id'),
            m.get('team_score_b', {}).get('team', {}).get('name'),
            m.get('team_score_b', {}).get('team', {}).get('short_name'),
            m.get('team_score_b', {}).get('score', [])[0].get('score'),
            m.get('match_outcome', {}).get('outcome'),
            m.get('match_outcome', {}).get('result_type'),
            m.get('match_outcome', {}).get('winner', {}).get('team_reference', {}).get('id'),
            m.get('match_outcome', {}).get('winner', {}).get('name'),
            m.get('match_outcome', {}).get('winner', {}).get('short_name'),
            m.get('half_time_score', {}).get('home'),
            m.get('half_time_score', {}).get('away'),
            m.get('full_time_score', {}).get('home'),
            m.get('full_time_score', {}).get('away'),
            m.get('clock')
        ]
    )

    homegoals = m.get('homeGoals', [])
    for g in homegoals:
        player = g.get('team_player')
        goal_id = g.get('id')
        for goal in g.get('goal', []):
            goals_data.append(
                [
                    match_ref, match_date, match_time,
                    player, goal_id,
                    goal.get('type'),
                    goal.get('time'),
                    goal.get('event_id'),
                    goal.get('event_time'),
                    'home'
                ]
            )

    awaygoals = m.get('awayGoals', [])
    for g in awaygoals:
        player = g.get('team_player')
        goal_id = g.get('id')
        for goal in g.get('goal', []):
            goals_data.append(
                [
                    match_ref, match_date, match_time,
                    player, goal_id,
                    goal.get('type'),
                    goal.get('time'),
                    goal.get('event_id'),
                    goal.get('event_time'),
                    'away'
                ]
            )

In [5]:
matches_cols = [
    'match_ref', 'match_date', 'match_time', 'state', 'match_type', 'competition_id', 'competition_name', 
    'round', 'legs', 'leg', 'team_a_id', 'team_a_name', 'team_a_short_name', 'team_a_score',
    'team_b_id', 'team_b_name', 'team_b_short_name', 'team_b_score',
    'outcome', 'result_type', 'winner_id', 'winner_name', 'winner_short_name',
    'half_time_score_home', 'half_time_score_away',
    'full_time_score_home', 'full_time_score_away', 'clock'
]
matches_df = pd.DataFrame(matches_data, columns=matches_cols)

In [6]:
goals_cols = [
    'match_ref', 'match_date', 'match_time', 'player', 'goal_id', 'type', 'time', 'event_id', 'event_time', 'side'
]
goals_df = pd.DataFrame(goals_data, columns=goals_cols)

In [7]:
def get_extra_time(t):
    time_split = t.replace("'", "").split("+")
    if len(time_split)>1:
        return int(time_split[1])
    else:
        return 0

goals_df['time_regular'] = goals_df['time'].apply(lambda x: int(x.replace("'", "").split("+")[0]))
goals_df['time_extra'] = goals_df['time'].apply(get_extra_time)

## Generate Features and Model

In [8]:
time_cutoffs = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90]
time_features = []
for i, t in enumerate(time_cutoffs):
    goals_df[f'goals_pre_{t}m'] = (goals_df[f'time_regular']<t)*1
    time_features += [f'goals_pre_{t}m']
    if t > min(time_cutoffs):
        goals_df[f'goals_{time_cutoffs[i-1]}m_to_{t}m'] = (
            goals_df[f'goals_pre_{t}m'] - goals_df[f'goals_pre_{time_cutoffs[i-1]}m'])
        time_features += [f'goals_{time_cutoffs[i-1]}m_to_{t}m']
    
    goals_df[f'goals_post_{t}m'] = (goals_df[f'time_regular']>=t)*1
    time_features += [f'goals_post_{t}m']

In [9]:
goals_features = goals_df.groupby(['match_ref', 'match_date'])[time_features].sum().reset_index()

In [21]:
model_data = matches_df.merge(goals_features, how='left', on=['match_ref', 'match_date'])

for f in time_features:
    model_data[f] = model_data[f].fillna(0)

In [23]:
def get_minutes(c):
    try:
        if c == 'HT':
            return 45
        else:
            return int(c.replace("'", "").split("+")[0])
    except:
        return None

model_data['minutes_of_play'] = model_data['clock'].apply(get_minutes)

In [24]:
matches_df['minutes_of_play'] = matches_df['clock'].apply(get_minutes)

In [25]:
#viable_matches = matches_df[matches_df['minutes_of_play']<=80]
viable_matches = model_data[model_data['minutes_of_play']<=80]

In [26]:
pd.options.display.max_columns = 100
viable_matches

,match_ref,match_date,match_time,state,match_type,competition_id,competition_name,round,legs,leg,team_a_id,team_a_name,team_a_short_name,team_a_score,team_b_id,team_b_name,team_b_short_name,team_b_score,outcome,result_type,winner_id,winner_name,winner_short_name,half_time_score_home,half_time_score_away,full_time_score_home,full_time_score_away,clock,minutes_of_play,goals_pre_5m,goals_post_5m,goals_pre_10m,goals_5m_to_10m,goals_post_10m,goals_pre_15m,goals_10m_to_15m,goals_post_15m,goals_pre_20m,goals_15m_to_20m,goals_post_20m,goals_pre_25m,goals_20m_to_25m,goals_post_25m,goals_pre_30m,goals_25m_to_30m,goals_post_30m,goals_pre_35m,goals_30m_to_35m,goals_post_35m,goals_pre_40m,goals_35m_to_40m,goals_post_40m,goals_pre_45m,goals_40m_to_45m,goals_post_45m,goals_pre_50m,goals_45m_to_50m,goals_post_50m,goals_pre_55m,goals_50m_to_55m,goals_post_55m,goals_pre_60m,goals_55m_to_60m,goals_post_60m,goals_pre_65m,goals_60m_to_65m,goals_post_65m,goals_pre_70m,goals_65m_to_70m,goals_post_70m,goals_pre_75m,goals_70m_to_75m,goals_post_75m,goals_pre_80m,goals_75m_to_80m,goals_post_80m,goals_pre_85m,goals_80m_to_85m,goals_post_85m,goals_pre_90m,goals_85m_to_90m,goals_post_90m
5,150385,2022-12-31T14:30:00.000Z,14:30,SECONDHALF,Regular,54,Welsh Premier League,{},1,1,3302,Flint Town United,None,0,843,Connah's Quay Nomads,None,0,None,None,NaN,None,None,NaN,NaN,NaN,NaN,80',80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,150409,2022-12-31T14:30:00.000Z,14:30,SECONDHALF,Regular,54,Welsh Premier League,{},1,1,968,Airbus UK Broughton,None,0,729,The New Saints,TNS,8,None,None,NaN,None,None,NaN,NaN,NaN,NaN,80',80.0,1.0,7.0,1.0,0.0,7.0,1.0,0.0,7.0,3.0,2.0,5.0,3.0,0.0,5.0,3.0,0.0,5.0,4.0,1.0,4.0,4.0,0.0,4.0,6.0,2.0,2.0,6.0,0.0,2.0,7.0,1.0,1.0,7.0,0.0,1.0,7.0,0.0,1.0,8.0,1.0,0.0,8.0,0.0,0.0,8.0,0.0,0.0,8.0,0.0,0.0,8.0,0.0,0.0
7,150425,2022-12-31T14:30:00.000Z,14:30,SECONDHALF,Regular,54,Welsh Premier League,{},1,1,3301,Haverfordwest County,None,0,4001,Pontypridd United,Pontypridd,1,None,None,NaN,None,None,NaN,NaN,NaN,NaN,80',80.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
8,145085,2022-12-31T15:00:00.000Z,15:00,SECONDHALF,Regular,1,English Premier League,{},1,1,15,Bournemouth,Bournemouth,0,3,Crystal Palace,C Palace,2,None,None,NaN,None,None,NaN,NaN,NaN,NaN,54',54.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,2.0,1.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0
9,145090,2022-12-31T15:00:00.000Z,15:00,SECONDHALF,Regular,1,English Premier League,{},1,1,208,Fulham,Fulham,1,18,Southampton,Southampton,0,None,None,NaN,None,None,NaN,NaN,NaN,NaN,51',51.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
10,145083,2022-12-31T15:00:00.000Z,15:00,SECONDHALF,Regular,1,English Premier League,{},1,1,13,Manchester City,Man City,1,5,Everton,Everton,0,None,None,NaN,None,None,NaN,NaN,NaN,NaN,50',50.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
11,145813,2022-12-31T15:00:00.000Z,15:00,SECONDHALF,Regular,59,Scottish League Two,{},1,1,726,Stirling Albion,Stirling,2,713,Stenhousemuir,Stenhousemuir,1,None,None,NaN,None,None,NaN,NaN,NaN,NaN,54',54.0,0.0,3.0,0.0,0.0,3.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,0.0,1.0,3.0,1.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,3.

# Betfair Live Data

## Sign in

In [27]:
try:

    header = {'X-Application': application, 'Content-Type': 'application/x-www-form-urlencoded'}
    auth = 'username='+username+'&password='+password

    login = requests.post('https://identitysso-cert.betfair.com/api/certlogin',
                          cert=('/etc/ssl/client-2048.crt', '/etc/ssl/client-2048.key'),
                          headers=header, data=auth)
    
    login_success = login.json()['loginStatus']
    #logging.info('Login '+str(login_success))

except Exception as error:
    login_status_code = login.status_code
    #logging.error('Login error '+str(login_status_code))
    
    # after logging will want to either pass (continue with code below), continue (go back to start of loop perhaps with a pause first) or break

In [28]:
login.json()

{'sessionToken': '2jzgkslf+Cvu8UjZEsiJgxsV40iXASfiLMKXcpSmUwA=',
 'loginStatus': 'SUCCESS'}

In [29]:
login.status_code

200

In [30]:
ssoid = login.json()['sessionToken']

## Get common details

In [31]:
bet_url = "https://api.betfair.com/exchange/betting/json-rpc/v1"
headers = {'X-Application': application, 'X-Authentication': ssoid, 'content-type': 'application/json'}
# event_req = '{"jsonrpc": "2.0", "method": "SportsAPING/v1.0/listEventTypes", "params": {"filter":{ }}, "id": 1}'
# request = requests.post(bet_url, data=event_req.encode('utf-8'), headers=headers) 
# request.json()

## Get Events

In [32]:
events = []

event_type_id = '["1"]'
market_start_time = (datetime.datetime.now() + datetime.timedelta(hours=-10)).strftime('%Y-%m-%dT%H:%M:%SZ')
market_end_time = (datetime.datetime.now() + datetime.timedelta(hours=24)).strftime('%Y-%m-%dT%H:%M:%SZ')
inplay = 'true'

user_req='{"jsonrpc": "2.0", "method": "SportsAPING/v1.0/listEvents",\
           "params": {"filter":{"eventTypeIds":'+event_type_id+',\
           "inPlayOnly":'+inplay+', \
           "marketStartTime":{"from":"'+market_start_time+'", "to":"'+market_end_time+'"}}}, "id": 1}'

request = requests.post(bet_url, data=user_req.encode('utf-8'), headers=headers)
events += request.json()['result']

In [36]:
len(events)

26

In [34]:
events_df = pd.DataFrame([[e['event']['id'], e['event']['name']] for e in events], columns=['betfair_id', 'betfair_name'])

In [35]:
events_df

,betfair_id,betfair_name
0,31992361,Sekhukhune United v Amazulu
1,31992233,Azam FC v Mbeya City FC
2,31993697,Flint Town United v Connahs Quay
3,31954030,Bournemouth v Crystal Palace
4,31993698,Haverfordwest County v Pontypridd Town
5,31986787,Queen of South v Clyde
6,31993701,Airbus UK Broughton v The New Saints
7,31992699,Avion Academy FC v Aigle Royal de la Menoua
8,31992700,Renaissance de Ngoumou v PWD Bamenda
9,31954035,Man City v Everton


In [37]:
viable_matches['sporting_life_event_name'] = viable_matches['team_a_name'] + ' v ' + viable_matches['team_b_name']

/home/angus/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [38]:
events_df['sporting_life_event_name'] = None
events_df['match_ref'] = None

lev_max = 10
for i, n in zip(viable_matches['match_ref'], viable_matches['sporting_life_event_name']):
    for idx, row in events_df.iterrows():
        if levenshtein_distance(row['betfair_name'], n) < lev_max:
            events_df.at[idx, 'sporting_life_event_name'] = n
            events_df.at[idx, 'match_ref'] = i

In [39]:
events_df

,betfair_id,betfair_name,sporting_life_event_name,match_ref
0,31992361,Sekhukhune United v Amazulu,None,None
1,31992233,Azam FC v Mbeya City FC,None,None
2,31993697,Flint Town United v Connahs Quay,Flint Town United v Connah's Quay Nomads,150385
3,31954030,Bournemouth v Crystal Palace,Bournemouth v Crystal Palace,145085
4,31993698,Haverfordwest County v Pontypridd Town,Haverfordwest County v Pontypridd United,150425
5,31986787,Queen of South v Clyde,Queen of the South v Clyde,145411
6,31993701,Airbus UK Broughton v The New Saints,Airbus UK Broughton v The New Saints,150409
7,31992699,Avion Academy FC v Aigle Royal de la Menoua,None,None
8,31992700,Renaissance de Ngoumou v PWD Bamenda,None,None
9,31954035,Man City v Everton,Manchester City v Everton,145083


In [47]:
for idx, row in events_df.iterrows():
    print(levenshtein_distance(row['betfair_name'], 'Real Sociedad v Osasuna'))

19
20
23
24
33
19
30
34
25
17
18
18
26
28
19
32
18
16
0
30
31
25
22
21
21
23


#### Check if viable matches have a betfair match

In [42]:
viable_matches_with_betfair_id = viable_matches.merge(events_df, how='left', on=['sporting_life_event_name', 'match_ref'])
viable_matches_with_betfair_id = viable_matches_with_betfair_id[viable_matches_with_betfair_id['betfair_id'].notnull()]

In [43]:
len(viable_matches_with_betfair_id)

10

In [44]:
viable_matches

,match_ref,match_date,match_time,state,match_type,competition_id,competition_name,round,legs,leg,team_a_id,team_a_name,team_a_short_name,team_a_score,team_b_id,team_b_name,team_b_short_name,team_b_score,outcome,result_type,winner_id,winner_name,winner_short_name,half_time_score_home,half_time_score_away,full_time_score_home,full_time_score_away,clock,minutes_of_play,goals_pre_5m,goals_post_5m,goals_pre_10m,goals_5m_to_10m,goals_post_10m,goals_pre_15m,goals_10m_to_15m,goals_post_15m,goals_pre_20m,goals_15m_to_20m,goals_post_20m,goals_pre_25m,goals_20m_to_25m,goals_post_25m,goals_pre_30m,goals_25m_to_30m,goals_post_30m,goals_pre_35m,goals_30m_to_35m,goals_post_35m,goals_pre_40m,goals_35m_to_40m,goals_post_40m,goals_pre_45m,goals_40m_to_45m,goals_post_45m,goals_pre_50m,goals_45m_to_50m,goals_post_50m,goals_pre_55m,goals_50m_to_55m,goals_post_55m,goals_pre_60m,goals_55m_to_60m,goals_post_60m,goals_pre_65m,goals_60m_to_65m,goals_post_65m,goals_pre_70m,goals_65m_to_70m,goals_post_70m,goals_pre_75m,goals_70m_to_75m,goals_post_75m,goals_pre_80m,goals_75m_to_80m,goals_post_80m,goals_pre_85m,goals_80m_to_85m,goals_post_85m,goals_pre_90m,goals_85m_to_90m,goals_post_90m,sporting_life_event_name
5,150385,2022-12-31T14:30:00.000Z,14:30,SECONDHALF,Regular,54,Welsh Premier League,{},1,1,3302,Flint Town United,None,0,843,Connah's Quay Nomads,None,0,None,None,NaN,None,None,NaN,NaN,NaN,NaN,80',80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Flint Town United v Connah's Quay Nomads
6,150409,2022-12-31T14:30:00.000Z,14:30,SECONDHALF,Regular,54,Welsh Premier League,{},1,1,968,Airbus UK Broughton,None,0,729,The New Saints,TNS,8,None,None,NaN,None,None,NaN,NaN,NaN,NaN,80',80.0,1.0,7.0,1.0,0.0,7.0,1.0,0.0,7.0,3.0,2.0,5.0,3.0,0.0,5.0,3.0,0.0,5.0,4.0,1.0,4.0,4.0,0.0,4.0,6.0,2.0,2.0,6.0,0.0,2.0,7.0,1.0,1.0,7.0,0.0,1.0,7.0,0.0,1.0,8.0,1.0,0.0,8.0,0.0,0.0,8.0,0.0,0.0,8.0,0.0,0.0,8.0,0.0,0.0,Airbus UK Broughton v The New Saints
7,150425,2022-12-31T14:30:00.000Z,14:30,SECONDHALF,Regular,54,Welsh Premier League,{},1,1,3301,Haverfordwest County,None,0,4001,Pontypridd United,Pontypridd,1,None,None,NaN,None,None,NaN,NaN,NaN,NaN,80',80.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,Haverfordwest County v Pontypridd United
8,145085,2022-12-31T15:00:00.000Z,15:00,SECONDHALF,Regular,1,English Premier League,{},1,1,15,Bournemouth,Bournemouth,0,3,Crystal Palace,C Palace,2,None,None,NaN,None,None,NaN,NaN,NaN,NaN,54',54.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,2.0,1.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,Bournemouth v Crystal Palace
9,145090,2022-12-31T15:00:00.000Z,15:00,SECONDHALF,Regular,1,English Premier League,{},1,1,208,Fulham,Fulham,1,18,Southampton,Southampton,0,None,None,NaN,None,None,NaN,NaN,NaN,NaN,51',51.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,Fulham v Southampton
10,145083,2022-12-31T15:00:00.000Z,15:00,SECONDHALF,Regular,1,English Premier League,{},1,1,13,Manchester City,Man City,1,5,Everton,Everton,0,None,None,NaN,None,None,NaN,NaN,NaN,NaN,50',50.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,Manchester City v Everton
11,145813,2022-12-31T15:00:00.000Z,15:00,SECONDHALF,Regular,59,Scottish League Two,{},1,1,726,Stirling Albion,Stirling,2,713,Stenhousemuir,Stenhousemu

In [45]:
viable_matches_with_betfair_id

,match_ref,match_date,match_time,state,match_type,competition_id,competition_name,round,legs,leg,team_a_id,team_a_name,team_a_short_name,team_a_score,team_b_id,team_b_name,team_b_short_name,team_b_score,outcome,result_type,winner_id,winner_name,winner_short_name,half_time_score_home,half_time_score_away,full_time_score_home,full_time_score_away,clock,minutes_of_play,goals_pre_5m,goals_post_5m,goals_pre_10m,goals_5m_to_10m,goals_post_10m,goals_pre_15m,goals_10m_to_15m,goals_post_15m,goals_pre_20m,goals_15m_to_20m,goals_post_20m,goals_pre_25m,goals_20m_to_25m,goals_post_25m,goals_pre_30m,goals_25m_to_30m,goals_post_30m,goals_pre_35m,goals_30m_to_35m,goals_post_35m,goals_pre_40m,goals_35m_to_40m,goals_post_40m,goals_pre_45m,goals_40m_to_45m,goals_post_45m,goals_pre_50m,goals_45m_to_50m,goals_post_50m,goals_pre_55m,goals_50m_to_55m,goals_post_55m,goals_pre_60m,goals_55m_to_60m,goals_post_60m,goals_pre_65m,goals_60m_to_65m,goals_post_65m,goals_pre_70m,goals_65m_to_70m,goals_post_70m,goals_pre_75m,goals_70m_to_75m,goals_post_75m,goals_pre_80m,goals_75m_to_80m,goals_post_80m,goals_pre_85m,goals_80m_to_85m,goals_post_85m,goals_pre_90m,goals_85m_to_90m,goals_post_90m,sporting_life_event_name,betfair_id,betfair_name
0,150385,2022-12-31T14:30:00.000Z,14:30,SECONDHALF,Regular,54,Welsh Premier League,{},1,1,3302,Flint Town United,None,0,843,Connah's Quay Nomads,None,0,None,None,NaN,None,None,NaN,NaN,NaN,NaN,80',80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Flint Town United v Connah's Quay Nomads,31993697,Flint Town United v Connahs Quay
1,150409,2022-12-31T14:30:00.000Z,14:30,SECONDHALF,Regular,54,Welsh Premier League,{},1,1,968,Airbus UK Broughton,None,0,729,The New Saints,TNS,8,None,None,NaN,None,None,NaN,NaN,NaN,NaN,80',80.0,1.0,7.0,1.0,0.0,7.0,1.0,0.0,7.0,3.0,2.0,5.0,3.0,0.0,5.0,3.0,0.0,5.0,4.0,1.0,4.0,4.0,0.0,4.0,6.0,2.0,2.0,6.0,0.0,2.0,7.0,1.0,1.0,7.0,0.0,1.0,7.0,0.0,1.0,8.0,1.0,0.0,8.0,0.0,0.0,8.0,0.0,0.0,8.0,0.0,0.0,8.0,0.0,0.0,Airbus UK Broughton v The New Saints,31993701,Airbus UK Broughton v The New Saints
2,150425,2022-12-31T14:30:00.000Z,14:30,SECONDHALF,Regular,54,Welsh Premier League,{},1,1,3301,Haverfordwest County,None,0,4001,Pontypridd United,Pontypridd,1,None,None,NaN,None,None,NaN,NaN,NaN,NaN,80',80.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,Haverfordwest County v Pontypridd United,31993698,Haverfordwest County v Pontypridd Town
3,145085,2022-12-31T15:00:00.000Z,15:00,SECONDHALF,Regular,1,English Premier League,{},1,1,15,Bournemouth,Bournemouth,0,3,Crystal Palace,C Palace,2,None,None,NaN,None,None,NaN,NaN,NaN,NaN,54',54.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,2.0,1.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,Bournemouth v Crystal Palace,31954030,Bournemouth v Crystal Palace
4,145090,2022-12-31T15:00:00.000Z,15:00,SECONDHALF,Regular,1,English Premier League,{},1,1,208,Fulham,Fulham,1,18,Southampton,Southampton,0,None,None,NaN,None,None,NaN,NaN,NaN,NaN,51',51.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,Fulham v Southampton,31954397,Fulham v Southampton
5,145083,2022-12-31T15:00:00.000Z,15:00,SECONDHALF,Regular,1,English Premier League,{},1,1,13,Manchester City,Man City,1,5,Everton,Everton,0,None,None,NaN,None,None,NaN,NaN,NaN,NaN,50',50.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0

#### Do preds

In [48]:
viable_matches_with_betfair_id['total_goals'] = (viable_matches_with_betfair_id['team_a_score'] + viable_matches_with_betfair_id['team_b_score']).astype(int)
viable_matches_with_betfair_id['market'] = 'Over/Under ' + viable_matches_with_betfair_id['total_goals'].astype(str) + '.5 Goals'
viable_matches_with_betfair_id['market_type'] = 'OVER_UNDER_' + viable_matches_with_betfair_id['total_goals'].astype(str) + '5'
viable_matches_with_betfair_id['market_id'] = None

viable_matches_with_betfair_id['pred_prob_over'] = 0.4
viable_matches_with_betfair_id['pred_odds_over'] = 1/viable_matches_with_betfair_id['pred_prob_over']
viable_matches_with_betfair_id['runner_name_over'] = 'Over ' + viable_matches_with_betfair_id['total_goals'].astype(str) + '.5 Goals'
viable_matches_with_betfair_id['selection_id_over'] = None
viable_matches_with_betfair_id['actual_odds_over_back_1'] = None
viable_matches_with_betfair_id['size_over_back_1'] = None
viable_matches_with_betfair_id['actual_odds_over_back_2'] = None
viable_matches_with_betfair_id['size_over_back_2'] = None
viable_matches_with_betfair_id['actual_odds_over_back_3'] = None
viable_matches_with_betfair_id['size_over_back_3'] = None

viable_matches_with_betfair_id['pred_prob_under'] = 1 - viable_matches_with_betfair_id['pred_prob_over']
viable_matches_with_betfair_id['pred_odds_under'] = 1/viable_matches_with_betfair_id['pred_prob_under']
viable_matches_with_betfair_id['runner_name_under'] = 'Under ' + viable_matches_with_betfair_id['total_goals'].astype(str) + '.5 Goals'
viable_matches_with_betfair_id['selection_id_under'] = None
viable_matches_with_betfair_id['actual_odds_under'] = None
viable_matches_with_betfair_id['actual_odds_under_back_1'] = None
viable_matches_with_betfair_id['size_under_back_1'] = None
viable_matches_with_betfair_id['actual_odds_under_back_2'] = None
viable_matches_with_betfair_id['size_under_back_2'] = None
viable_matches_with_betfair_id['actual_odds_under_back_3'] = None
viable_matches_with_betfair_id['size_under_back_3'] = None

In [49]:
viable_matches_with_betfair_id

,match_ref,match_date,match_time,state,match_type,competition_id,competition_name,round,legs,leg,team_a_id,team_a_name,team_a_short_name,team_a_score,team_b_id,team_b_name,team_b_short_name,team_b_score,outcome,result_type,winner_id,winner_name,winner_short_name,half_time_score_home,half_time_score_away,full_time_score_home,full_time_score_away,clock,minutes_of_play,goals_pre_5m,goals_post_5m,goals_pre_10m,goals_5m_to_10m,goals_post_10m,goals_pre_15m,goals_10m_to_15m,goals_post_15m,goals_pre_20m,goals_15m_to_20m,goals_post_20m,goals_pre_25m,goals_20m_to_25m,goals_post_25m,goals_pre_30m,goals_25m_to_30m,goals_post_30m,goals_pre_35m,goals_30m_to_35m,goals_post_35m,goals_pre_40m,...,goals_post_55m,goals_pre_60m,goals_55m_to_60m,goals_post_60m,goals_pre_65m,goals_60m_to_65m,goals_post_65m,goals_pre_70m,goals_65m_to_70m,goals_post_70m,goals_pre_75m,goals_70m_to_75m,goals_post_75m,goals_pre_80m,goals_75m_to_80m,goals_post_80m,goals_pre_85m,goals_80m_to_85m,goals_post_85m,goals_pre_90m,goals_85m_to_90m,goals_post_90m,sporting_life_event_name,betfair_id,betfair_name,total_goals,market,market_type,market_id,pred_prob_over,pred_odds_over,runner_name_over,selection_id_over,actual_odds_over_back_1,size_over_back_1,actual_odds_over_back_2,size_over_back_2,actual_odds_over_back_3,size_over_back_3,pred_prob_under,pred_odds_under,runner_name_under,selection_id_under,actual_odds_under,actual_odds_under_back_1,size_under_back_1,actual_odds_under_back_2,size_under_back_2,actual_odds_under_back_3,size_under_back_3
0,150385,2022-12-31T14:30:00.000Z,14:30,SECONDHALF,Regular,54,Welsh Premier League,{},1,1,3302,Flint Town United,None,0,843,Connah's Quay Nomads,None,0,None,None,NaN,None,None,NaN,NaN,NaN,NaN,80',80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Flint Town United v Connah's Quay Nomads,31993697,Flint Town United v Connahs Quay,0,Over/Under 0.5 Goals,OVER_UNDER_05,None,0.4,2.5,Over 0.5 Goals,None,None,None,None,None,None,None,0.6,1.666667,Under 0.5 Goals,None,None,None,None,None,None,None,None
1,150409,2022-12-31T14:30:00.000Z,14:30,SECONDHALF,Regular,54,Welsh Premier League,{},1,1,968,Airbus UK Broughton,None,0,729,The New Saints,TNS,8,None,None,NaN,None,None,NaN,NaN,NaN,NaN,80',80.0,1.0,7.0,1.0,0.0,7.0,1.0,0.0,7.0,3.0,2.0,5.0,3.0,0.0,5.0,3.0,0.0,5.0,4.0,1.0,4.0,4.0,...,1.0,7.0,0.0,1.0,7.0,0.0,1.0,8.0,1.0,0.0,8.0,0.0,0.0,8.0,0.0,0.0,8.0,0.0,0.0,8.0,0.0,0.0,Airbus UK Broughton v The New Saints,31993701,Airbus UK Broughton v The New Saints,8,Over/Under 8.5 Goals,OVER_UNDER_85,None,0.4,2.5,Over 8.5 Goals,None,None,None,None,None,None,None,0.6,1.666667,Under 8.5 Goals,None,None,None,None,None,None,None,None
2,150425,2022-12-31T14:30:00.000Z,14:30,SECONDHALF,Regular,54,Welsh Premier League,{},1,1,3301,Haverfordwest County,None,0,4001,Pontypridd United,Pontypridd,1,None,None,NaN,None,None,NaN,NaN,NaN,NaN,80',80.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,Haverfordwest County v Pontypridd United,31993698,Haverfordwest County v Pontypridd Town,1,Over/Under 1.5 Goals,OVER_UNDER_15,None,0.4,2.5,Over 1.5 Goals,None,None,None,None,None,None,None,0.6,1.666667,Under 1.5 Goals,None,None,None,None,None,None,None,None
3,145085,2022-12-31T15:00:00.000Z,15:00,SECONDHALF,Regular,1,English Premier League,{},1,1,15,Bournemouth,Bournemouth,0,3,Crystal Palace,C Palace,2,None,None,NaN,None,None,NaN,NaN,NaN,NaN,54',54.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,2.0,...,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,Bournemouth v Crystal Palace,31954030,Bournemouth v Crystal Palace,2,Over/Under 2.5 Goals,OVER_UNDER_25,None,0.4,2.5,Over 2.5 Goals,None,None,None,None,None,None,None,0.6,1.666667,Under 2.5 Goals,None,None,None,No

#### Get betfair odds

In [50]:
# markets
market_catalogue = []
for idx, row in viable_matches_with_betfair_id.iterrows():
    
    event_type_id = '["1"]'
    match_event_id = '["'+row['betfair_id']+'"]'
    market_types = '["'+row['market_type']+'"]'
    market_start_time = (datetime.datetime.now() + datetime.timedelta(hours=-24)).strftime('%Y-%m-%dT%H:%M:%SZ')
    market_end_time = (datetime.datetime.now() + datetime.timedelta(hours=24)).strftime('%Y-%m-%dT%H:%M:%SZ')
    max_results = str(200)
    sort_type = 'FIRST_TO_START'
    metadata = '["EVENT_TYPE", "COMPETITION", "EVENT", "MARKET_START_TIME", "MARKET_DESCRIPTION", "RUNNER_DESCRIPTION"]' #, "RUNNER_METADATA"]'
    inplay = 'true'

    user_req='{"jsonrpc": "2.0", "method": "SportsAPING/v1.0/listMarketCatalogue",\
               "params": {"filter":{"eventTypeIds":'+event_type_id+',"marketTypeCodes":'+market_types+',\
               "inPlayOnly":'+inplay+', "eventIds":'+match_event_id+',  \
               "marketStartTime":{"from":"'+market_start_time+'", "to":"'+market_end_time+'"}},\
               "sort":"'+sort_type+'", "maxResults":"'+max_results+'", "marketProjection":'+metadata+'}, "id": 1}'

    request = requests.post(bet_url, data=user_req.encode('utf-8'), headers=headers)
    
    try:
        request_result = request.json()['result'][0]
        viable_matches_with_betfair_id.at[idx, 'market_id'] = request_result['marketId']
        for s in request_result.get('runners', []):
            if s.get('runnerName') == row['runner_name_over']:
                viable_matches_with_betfair_id.at[idx, 'selection_id_over'] = s.get('selectionId')
            if s.get('runnerName') == row['runner_name_under']:
                viable_matches_with_betfair_id.at[idx, 'selection_id_under'] = s.get('selectionId')
    except:
        pass
    
    market_catalogue += request.json()['result']


In [51]:
viable_matches_with_betfair_id

,match_ref,match_date,match_time,state,match_type,competition_id,competition_name,round,legs,leg,team_a_id,team_a_name,team_a_short_name,team_a_score,team_b_id,team_b_name,team_b_short_name,team_b_score,outcome,result_type,winner_id,winner_name,winner_short_name,half_time_score_home,half_time_score_away,full_time_score_home,full_time_score_away,clock,minutes_of_play,goals_pre_5m,goals_post_5m,goals_pre_10m,goals_5m_to_10m,goals_post_10m,goals_pre_15m,goals_10m_to_15m,goals_post_15m,goals_pre_20m,goals_15m_to_20m,goals_post_20m,goals_pre_25m,goals_20m_to_25m,goals_post_25m,goals_pre_30m,goals_25m_to_30m,goals_post_30m,goals_pre_35m,goals_30m_to_35m,goals_post_35m,goals_pre_40m,...,goals_post_55m,goals_pre_60m,goals_55m_to_60m,goals_post_60m,goals_pre_65m,goals_60m_to_65m,goals_post_65m,goals_pre_70m,goals_65m_to_70m,goals_post_70m,goals_pre_75m,goals_70m_to_75m,goals_post_75m,goals_pre_80m,goals_75m_to_80m,goals_post_80m,goals_pre_85m,goals_80m_to_85m,goals_post_85m,goals_pre_90m,goals_85m_to_90m,goals_post_90m,sporting_life_event_name,betfair_id,betfair_name,total_goals,market,market_type,market_id,pred_prob_over,pred_odds_over,runner_name_over,selection_id_over,actual_odds_over_back_1,size_over_back_1,actual_odds_over_back_2,size_over_back_2,actual_odds_over_back_3,size_over_back_3,pred_prob_under,pred_odds_under,runner_name_under,selection_id_under,actual_odds_under,actual_odds_under_back_1,size_under_back_1,actual_odds_under_back_2,size_under_back_2,actual_odds_under_back_3,size_under_back_3
0,150385,2022-12-31T14:30:00.000Z,14:30,SECONDHALF,Regular,54,Welsh Premier League,{},1,1,3302,Flint Town United,None,0,843,Connah's Quay Nomads,None,0,None,None,NaN,None,None,NaN,NaN,NaN,NaN,80',80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Flint Town United v Connah's Quay Nomads,31993697,Flint Town United v Connahs Quay,0,Over/Under 0.5 Goals,OVER_UNDER_05,1.208130923,0.4,2.5,Over 0.5 Goals,5851483,None,None,None,None,None,None,0.6,1.666667,Under 0.5 Goals,5851482,None,None,None,None,None,None,None
1,150409,2022-12-31T14:30:00.000Z,14:30,SECONDHALF,Regular,54,Welsh Premier League,{},1,1,968,Airbus UK Broughton,None,0,729,The New Saints,TNS,8,None,None,NaN,None,None,NaN,NaN,NaN,NaN,80',80.0,1.0,7.0,1.0,0.0,7.0,1.0,0.0,7.0,3.0,2.0,5.0,3.0,0.0,5.0,3.0,0.0,5.0,4.0,1.0,4.0,4.0,...,1.0,7.0,0.0,1.0,7.0,0.0,1.0,8.0,1.0,0.0,8.0,0.0,0.0,8.0,0.0,0.0,8.0,0.0,0.0,8.0,0.0,0.0,Airbus UK Broughton v The New Saints,31993701,Airbus UK Broughton v The New Saints,8,Over/Under 8.5 Goals,OVER_UNDER_85,1.208130736,0.4,2.5,Over 8.5 Goals,2407529,None,None,None,None,None,None,0.6,1.666667,Under 8.5 Goals,2407528,None,None,None,None,None,None,None
2,150425,2022-12-31T14:30:00.000Z,14:30,SECONDHALF,Regular,54,Welsh Premier League,{},1,1,3301,Haverfordwest County,None,0,4001,Pontypridd United,Pontypridd,1,None,None,NaN,None,None,NaN,NaN,NaN,NaN,80',80.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,Haverfordwest County v Pontypridd United,31993698,Haverfordwest County v Pontypridd Town,1,Over/Under 1.5 Goals,OVER_UNDER_15,None,0.4,2.5,Over 1.5 Goals,None,None,None,None,None,None,None,0.6,1.666667,Under 1.5 Goals,None,None,None,None,None,None,None,None
3,145085,2022-12-31T15:00:00.000Z,15:00,SECONDHALF,Regular,1,English Premier League,{},1,1,15,Bournemouth,Bournemouth,0,3,Crystal Palace,C Palace,2,None,None,NaN,None,None,NaN,NaN,NaN,NaN,54',54.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,2.0,...,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,Bournemouth v Crystal Palace,31954030,Bournemouth v Crystal Palace,2,Over/Under 2.5 Goals,OVER_UNDER_25,1.207324670,0.4,2.5,Over 2.5 Goals,47973,None,None,None,None,None,None,0.6,1.666667

In [52]:
market_catalogue

[{'marketId': '1.208130923',
  'marketName': 'Over/Under 0.5 Goals',
  'marketStartTime': '2022-12-31T14:32:24.000Z',
  'description': {'persistenceEnabled': True,
   'bspMarket': False,
   'marketTime': '2022-12-31T14:32:24.000Z',
   'suspendTime': '2022-12-31T14:32:24.000Z',
   'bettingType': 'ODDS',
   'turnInPlayEnabled': True,
   'marketType': 'OVER_UNDER_05',
   'regulator': 'GIBRALTAR REGULATOR',
   'marketBaseRate': 2.0,
   'discountAllowed': False,
   'wallet': 'UK wallet',
   'rules': '<!--Football - Over/Unders --><br>How many goals will be scored in this match.<br> All bets apply to Full Time according to the match officials, plus any stoppage time. Extra-time/penalty shoot-outs are not included.<br><br></b>For further information please see <a href=http://content.betfair.com/aboutus/content.asp?sWhichKey=Rules%20and%20Regulations#undefined.do style=color:0163ad; text-decoration: underline; target=_blank>Rules & Regs<br><br>\n',
   'rulesHasDate': True,
   'priceLadderDescr

#### Get odds for each market

In [53]:
market_books = []
for idx, row in viable_matches_with_betfair_id.iterrows():
    
    priceProjection = '["EX_BEST_OFFERS"]'
    prices_req = '{"jsonrpc": "2.0", "method": "SportsAPING/v1.0/listMarketBook", "params": {"marketIds": ["' + str(row['market_id']) + '"],"priceProjection":{"priceData":["EX_BEST_OFFERS"]}}, "id": 1}'
    request = requests.post(bet_url, data=prices_req.encode('utf-8'), headers=headers)
    prices_result = request.json()
    
    try:
        prices_dict = prices_result['result'][0]
        runners = prices_dict.get('runners', [])
        
        for r in runners:
            
            if r['selectionId'] == row['selection_id_over']:
                for i, a in enumerate(r.get('ex', {}).get('availableToBack', [])):
                    viable_matches_with_betfair_id.at[idx, f'actual_odds_over_back_{i+1}'] = a['price']
                    viable_matches_with_betfair_id.at[idx, f'size_over_back_{i+1}'] = a['size']
                for i, a in enumerate(r.get('ex', {}).get('availableToLay', [])):
                    viable_matches_with_betfair_id.at[idx, f'actual_odds_over_lay_{i+1}'] = a['price']
                    viable_matches_with_betfair_id.at[idx, f'size_over_lay_{i+1}'] = a['size']
            
            if r['selectionId'] == row['selection_id_under']:
                for i, a in enumerate(r.get('ex', {}).get('availableToBack', [])):
                    viable_matches_with_betfair_id.at[idx, f'actual_odds_under_back_{i+1}'] = a['price']
                    viable_matches_with_betfair_id.at[idx, f'size_under_back_{i+1}'] = a['size']
                for i, a in enumerate(r.get('ex', {}).get('availableToLay', [])):
                    viable_matches_with_betfair_id.at[idx, f'actual_odds_under_lay_{i+1}'] = a['price']
                    viable_matches_with_betfair_id.at[idx, f'size_under_lay_{i+1}'] = a['size']
        
    except:
        pass
    
    market_books.append(prices_dict)

In [57]:
prices_dict

{'marketId': '1.206292964',
 'isMarketDataDelayed': True,
 'status': 'OPEN',
 'betDelay': 5,
 'bspReconciled': False,
 'complete': True,
 'inplay': True,
 'numberOfWinners': 1,
 'numberOfRunners': 2,
 'numberOfActiveRunners': 2,
 'lastMatchTime': '2022-12-31T16:20:00.280Z',
 'totalMatched': 22231.84,
 'totalAvailable': 32684.51,
 'crossMatching': True,
 'runnersVoidable': False,
 'version': 4973661476,
 'runners': [{'selectionId': 1221385,
   'handicap': 0.0,
   'status': 'ACTIVE',
   'lastPriceTraded': 3.2,
   'totalMatched': 0.0,
   'ex': {'availableToBack': [{'price': 3.1, 'size': 134.74},
     {'price': 3.05, 'size': 2178.0},
     {'price': 3.0, 'size': 1697.07}],
    'availableToLay': [{'price': 3.2, 'size': 104.32},
     {'price': 3.25, 'size': 1191.95},
     {'price': 3.3, 'size': 374.28}],
    'tradedVolume': []}},
  {'selectionId': 1221386,
   'handicap': 0.0,
   'status': 'ACTIVE',
   'lastPriceTraded': 1.46,
   'totalMatched': 0.0,
   'ex': {'availableToBack': [{'price': 1.4

In [55]:
viable_matches_with_betfair_id

,match_ref,match_date,match_time,state,match_type,competition_id,competition_name,round,legs,leg,team_a_id,team_a_name,team_a_short_name,team_a_score,team_b_id,team_b_name,team_b_short_name,team_b_score,outcome,result_type,winner_id,winner_name,winner_short_name,half_time_score_home,half_time_score_away,full_time_score_home,full_time_score_away,clock,minutes_of_play,goals_pre_5m,goals_post_5m,goals_pre_10m,goals_5m_to_10m,goals_post_10m,goals_pre_15m,goals_10m_to_15m,goals_post_15m,goals_pre_20m,goals_15m_to_20m,goals_post_20m,goals_pre_25m,goals_20m_to_25m,goals_post_25m,goals_pre_30m,goals_25m_to_30m,goals_post_30m,goals_pre_35m,goals_30m_to_35m,goals_post_35m,goals_pre_40m,...,goals_post_75m,goals_pre_80m,goals_75m_to_80m,goals_post_80m,goals_pre_85m,goals_80m_to_85m,goals_post_85m,goals_pre_90m,goals_85m_to_90m,goals_post_90m,sporting_life_event_name,betfair_id,betfair_name,total_goals,market,market_type,market_id,pred_prob_over,pred_odds_over,runner_name_over,selection_id_over,actual_odds_over_back_1,size_over_back_1,actual_odds_over_back_2,size_over_back_2,actual_odds_over_back_3,size_over_back_3,pred_prob_under,pred_odds_under,runner_name_under,selection_id_under,actual_odds_under,actual_odds_under_back_1,size_under_back_1,actual_odds_under_back_2,size_under_back_2,actual_odds_under_back_3,size_under_back_3,actual_odds_under_lay_1,size_under_lay_1,actual_odds_under_lay_2,size_under_lay_2,actual_odds_under_lay_3,size_under_lay_3,actual_odds_over_lay_1,size_over_lay_1,actual_odds_over_lay_2,size_over_lay_2,actual_odds_over_lay_3,size_over_lay_3
0,150385,2022-12-31T14:30:00.000Z,14:30,SECONDHALF,Regular,54,Welsh Premier League,{},1,1,3302,Flint Town United,None,0,843,Connah's Quay Nomads,None,0,None,None,NaN,None,None,NaN,NaN,NaN,NaN,80',80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Flint Town United v Connah's Quay Nomads,31993697,Flint Town United v Connahs Quay,0,Over/Under 0.5 Goals,OVER_UNDER_05,1.208130923,0.4,2.5,Over 0.5 Goals,5851483,8.6,10.35,8,31.73,6.8,18.81,0.6,1.666667,Under 0.5 Goals,5851482,None,1.11,237.28,1.1,23.82,1.08,43.62,1.13,78.78,1.14,222.75,1.16,10.89,10.00,24.74,21.00,10.14,110.00,7.20
1,150409,2022-12-31T14:30:00.000Z,14:30,SECONDHALF,Regular,54,Welsh Premier League,{},1,1,968,Airbus UK Broughton,None,0,729,The New Saints,TNS,8,None,None,NaN,None,None,NaN,NaN,NaN,NaN,80',80.0,1.0,7.0,1.0,0.0,7.0,1.0,0.0,7.0,3.0,2.0,5.0,3.0,0.0,5.0,3.0,0.0,5.0,4.0,1.0,4.0,4.0,...,0.0,8.0,0.0,0.0,8.0,0.0,0.0,8.0,0.0,0.0,Airbus UK Broughton v The New Saints,31993701,Airbus UK Broughton v The New Saints,8,Over/Under 8.5 Goals,OVER_UNDER_85,1.208130736,0.4,2.5,Over 8.5 Goals,2407529,1.04,343.6,None,None,None,None,0.6,1.666667,Under 8.5 Goals,2407528,None,1.02,120.59,1.01,163.91,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,150425,2022-12-31T14:30:00.000Z,14:30,SECONDHALF,Regular,54,Welsh Premier League,{},1,1,3301,Haverfordwest County,None,0,4001,Pontypridd United,Pontypridd,1,None,None,NaN,None,None,NaN,NaN,NaN,NaN,80',80.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,Haverfordwest County v Pontypridd United,31993698,Haverfordwest County v Pontypridd Town,1,Over/Under 1.5 Goals,OVER_UNDER_15,None,0.4,2.5,Over 1.5 Goals,None,None,None,None,None,None,None,0.6,1.666667,Under 1.5 Goals,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,145085,2022-12-31T15:00:00.000Z,15:00,SECONDHALF,Regular,1,English Premier League,{},1,1,15,Bournemouth,Bournemouth,0,3,Crystal Palace,C Palace,2,None,None,NaN,None,None,NaN,NaN,NaN,NaN,54',54.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,2.0,...,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,Bournemouth v Crystal Palace,31954030,Bournemouth v Crystal Palace,2,Over/Under 2.5 Goals,OVER_UNDER_25,1.207324670,0.4,2.5,Over 2.5 Goals,47973,1.72,8

# Need to check not excluding 0-0 games in model building

# Newcastle vs Leeds didn't work because of 'united' so need to be cleverer about that